In [1]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include(pathtorepo * "dynamical-systems\\system.jl")


using StaticArrays, DifferentialEquations, DynamicalSystems, JLD
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using CairoMakie, GLMakie
using LinearAlgebra, LinearSolve

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

In [2]:
cd("C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\файлы matlab")
I0_hom = load("I0_hom_hom.jld")["data"]
u0_hom = load("U0_hom_hom.jld")["data"]
I0_hom = I0_hom[:]
u0_hom = u0_hom[:];

In [24]:
const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305

t = 1000.0; tt = 1000.0; tstep = 0.001; tspan = (1.0, t)
integ_set = (alg = Vern9(), adaptive = true, abstol = 1e-15, reltol = 1e-15);

In [25]:
index_hom = 1
I0 = I0_hom[index_hom]; U0 = u0_hom[index_hom]
p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]
u0 = [11.325905642223786, 0.6594706953104683, 0.4863175026548461];

In [26]:
ds = CoupledODEs(TM, u0, p, diffeq = integ_set)
fp, _, _ = fixedpoints(ds, box, jacob_TM_)
ϵ_sep = 1e-7
ϵ_box = 1e-10
Ju0 = jacob_TM_(fp[1], p, 0);

In [27]:
eigen_val_vec = eigen(Ju0)
eigen_vectors = eigen_val_vec.vectors

v1 = real(eigen_vectors[:, 1])
v2 = real(eigen_vectors[:, 2])
v3 = imag(eigen_vectors[:, 3])

v1 = normalize(v1, Inf)
v2 = normalize(v2, Inf)
v3 = normalize(v3, Inf)

v1t = transpose(v1)
v2t = transpose(v2)
v3t = transpose(v3)

dot_on_sep_stable = fp[1] + v1*ϵ_sep
A = transpose([v1t; v2t; v3t])
A = Matrix(A)

3×3 Matrix{Float64}:
  1.0         1.0          0.0
 -0.0490503  -0.0141828    1.0
  0.0119273   0.000705923  0.176983

In [28]:
function condition(u, p, t, integrator)
    x = Vector(u)
    b = x - fp[1]
    b = Vector(b)
    linprob = LinearProblem(A, b)
    linsolve = solve(linprob)
    return norm(linsolve.u, Inf)
end
affect!(integrator) = terminate!(integrator)

affect! (generic function with 1 method)

In [30]:
probODE = ODEProblem(TM, dot_on_sep_stable, tspan, p)

ODEProblem with uType SVector{3, Float64} and tType Float64. In-place: false
timespan: (1.0, 1000.0)
u0: 3-element SVector{3, Float64} with indices SOneTo(3):
 8.345808551620786
 0.7384946597477061
 0.43829856154578317

In [33]:
sol = solve(probODE, alg = Vern9(), adaptive = false, dt = tstep, )

retcode: Success
Interpolation: specialized 9th order lazy interpolation
t: 999002-element Vector{Float64}:
    1.0
    1.001
    1.0019999999999998
    1.0029999999999997
    1.0039999999999996
    1.0049999999999994
    1.0059999999999993
    1.0069999999999992
    1.0079999999999991
    1.008999999999999
    ⋮
  999.9929999832652
  999.9939999832652
  999.9949999832652
  999.9959999832652
  999.9969999832651
  999.9979999832651
  999.9989999832651
  999.9999999832651
 1000.0
u: 999002-element Vector{SVector{3, Float64}}:
 [8.345808551620786, 0.7384946597477061, 0.43829856154578317]
 [8.345808550983747, 0.7384946597789531, 0.438298561538185]
 [8.345808550350766, 0.7384946598100011, 0.4382985615306353]
 [8.345808549721816, 0.7384946598408512, 0.4382985615231337]
 [8.345808549096875, 0.7384946598715048, 0.4382985615156798]
 [8.345808548475913, 0.7384946599019631, 0.43829856150827345]
 [8.345808547858908, 0.7384946599322274, 0.4382985615009143]
 [8.345808547245834, 0.7384946599622989, 0

In [10]:
for (i, dot) in enumerate(tr)

    b = dot - fp[1]
    b = Vector(b)
    prob = LinearProblem(A, b)
    sol = solve(prob)
    
    """vec = [abs(sol.u[1]), abs(sol.u[2]), abs(sol.u[3])]
    norminf = maximum(vec)"""

    αs[:, i] = sol
    norminfs[i] = norm(sol.u, Inf)
end